약한 참조 - 레퍼런스 카운트로 고려되지 않는 참조
객체는 식별자를 통해 객체를 reference하면 reference 카운트가 존재
레퍼런스 카운터는 객체를 만들면 항상 증가
but,약한 참조 사용 시 객체의 레퍼런스 카운트를 증가시키지 않음
- 순환 참조가 방지됨(서로다른 객체들 사이에 참조 방식이 순환형태로 연결되는 방식, 독립적으로 존재하지만 순환 참조 되는 서로 다른 객체 그룹은 쓰레기 수집이 안됨)
  - 순환참조 쓰레기 수집 기능이 있지만 CPU자원 낭비 심함, 쓰레기 수집 빈도가 낮으면 순환참조되는 많은 객체들이 메모리를 쓸 데 없이 점유함
- 다양한 인스턴스들 사이에서 공유되는 객체에 대한 일종의 케시(Cache)를 만드는 데 활용됨

####weakref.ref(o)
- weakref 모듈의 ref(o) 함수

  - 객체 o에 대한 약한 참조를 생성한다.
  - 해당 객체가 메모리에 정상적으로 남아 있는지 조사한다.
    - 객체가 메모리에 남아 있지 않으면 None을 반환한다.
- 약한 참조로 부터 실제 객체를 참조하는 방법

  - 약한 참조 객체에 함수형태 호출

In [5]:
import sys
import weakref#weakref 모듈 임포트
class C:
    pass
c = C()#클래스 C의 인스턴스 생성 
c.a = 1#인스턴스 c에 테스트용 값 설정 
print "refcount -", sys.getrefcount(c)
#객체 c의 레퍼런스 카운트 조회
#c라는 식별자가 실제 객체를 가리키는 레퍼런스의 카운트가 1
#but 파이썬 내부적으로 보이지 않는 레퍼런스 존재
# -> 레퍼런스 카운트가 1이 되면 c 객체는 사라짐
#getrefcount()객체의 레퍼런스 카운트가 몇인지 알아보는 메소드
#sys는 getrefcount를 위해 import
#weakref 약한 참조를 만들 수 있는 모듈
print

d = c#일반적인 레퍼런스 카운트 증가 방법
#c의 레퍼런스 값을 d에 카피해줌 -> d의 레퍼런스 카운트 증가
print "refcount -", sys.getrefcount(c) 
#객체 c의 레퍼런스 카운트 조회
print 

r = weakref.ref(c)#약한 참조 객체 r 생성
print "refcount -", sys.getrefcount(c) 
#객체 c의 레퍼런스 카운트 조회 --> 카운트 불변
print

refcount - 2

refcount - 3

refcount - 3



In [6]:
#만든 약한 참조의 활용
print r#약한 참조(weakref) 객체
print r()#실제 약한 참조가 참조하고 있는 객체를 반환
#약한 참조로 부터 실제 객체를 참조하는 방법: 
# - >약한 참조 객체에 함수형태로 호출
print c 
print r().a#약한 참조를 이용한 실제 객체 멤버 참조
print 

del c#객체 제거
del d
#c와 d는 일반적인 레퍼런스 변수로 삭제하면 레퍼런스 카운트 감소
print r()#None을 리턴한다
#실제 약한 참조가 참조하고 있는 객체를 반환하는 건데 없어서 None
print r().a#속성도 참조할 수 없다

<weakref at 00000000039A4EA8; to 'instance' at 00000000039A7A88>
<__main__.C instance at 0x00000000039A7A88>
<__main__.C instance at 0x00000000039A7A88>
1

None


AttributeError: 'NoneType' object has no attribute 'a'

In [9]:
import weakref
#약한 참조는 파이썬이 기본적으로 제공하는 내장자료형 객체에는 사용 불가
d = {'one': 1, 'two': 2}
wd = weakref.ref(d)
#사용자가 정의한 클래스의 인스턴스에만 약한 참조 사용 가능

TypeError: cannot create weak reference to 'dict' object

weakref.proxy(o) -> 활용도가 좀 더 높음
- weakref의 proxy(o)는 객체 o에 대한 약한 참조 프록시를 생성한다.
  - 프록시를 이용하면 함수 형식을 사용하지 않아도 실제 객체를 바로 참조할 수 있다.
  - ref(o) 함수보다 더 선호되는 함수

In [10]:
import sys
import weakref
class C:
    pass

c = C()
c.a = 2
print "refcount -", sys.getrefcount(c) 
#객체 c의 레퍼런스 카운트 조회
p = weakref.proxy(c)#프록시 객체를 만든다->코딩량 줄이기 가능
print "refcount -", sys.getrefcount(c)
#객체 c의 레퍼런스 카운트 조회 --> 카운트 불변
print 
print p#실제로 p가 가리키는 실제 객체 호출
#근데 이게 왜 필요한지...
print c
print p.a
#객체 c,p는 동일한 객체지만 p는 weakref->???어차피 차이 딱히 없지 않나..?

refcount - 2
refcount - 2

<__main__.C instance at 0x00000000039A7A88>
<__main__.C instance at 0x00000000039A7A88>
2


In [12]:
import weakref
class C:
    pass

c = C() # 참조할 객체 생성
r = weakref.ref(c) # weakref 생성
p = weakref.proxy(c) # weakref 프록시 생성
print weakref.getweakrefcount(c) # weakref 개수 조회 => 2
#getweakrefcount() -> 약한 참조가 몇개 존재하는지 세는 함수
print weakref.getweakrefs(c) # weakref 목록 조회
#getweakrefs() -> 반환되는 리스트 안 원소가 약한 참조

2
[<weakref at 0000000003A24868; to 'instance' at 0000000003A30C48>, <weakproxy at 0000000003A249F8 to instance at 0000000003A30C48>]


- 약한 사전 (Weak Dictionary)
  - 사전의 키(key)나 값(value)으로 다른 객체들에 대한 약한 참조를 지니는 사전
  - 주로 다른 객체들에 대한 캐시(Cache)로 활용
  - 일반적인 사전과의 차이점
    - 키(key)나 값(value)으로 사용되는 객체는 약한 참조를 지닌다.
    - 실제 객체가 삭제되면 자동적으로 약한 사전에 있는 (키, 값)의 쌍도 삭제된다.!!!!!!(중요!!!!!!)
    - 즉, 실제 객체가 사라지면 캐시역할을 하는 약한 사전에서도 해당 아이템이 제거되므로 효율적인 객체 소멸 관리가 가능하다.(효율적인 메모리 관리 가능)
- weakref 모듈의 WeakValueDictionary 클래스
  - weakref 모듈의 WeakValueDictionary 클래스의 생성자는 약한 사전을 생성한다.

In [13]:
import weakref
class C:
    pass

c = C()
c.a = 4
d = weakref.WeakValueDictionary()
#클래스로 클래스의 생성자를 호출하는 것.WeakValueDictionary는 클래스임
#WeakValueDictionary라는 새로운 자료형의 객체를 d에 할당
#WeakValueDictionary 객체 생성
print d

d[1] = c#실제 객체에 대한 약한 참조 아이템 생성
#1이라는 key값의 value로 c를 넣을 수 있음
print d.items()#사전 내용 확인
#1이라는 키 값과 value...
print d[1].a#실제 객체의 속성 참조
#d[1]->사전의 검색 = c

del c#실제 객체 삭제, 레버런스 카운트가 1로 줄음
print d.items()#약한 사전에 해당 객체 아이템도 제거되어 있음

#일반적인 사전의 경우 del c를 하더라도 객체가 삭제되지 않음
#이유 : 사전 자체가 객체를 레퍼런스로 가지고 있음
#but 약한 사전은 레퍼런스를 약한 참조로 가지고 있어서 삭제 가능!

<WeakValueDictionary at 61017160>
[(1, <__main__.C instance at 0x0000000003A30CC8>)]
4
[]


In [14]:
#위 내용을 일반 사전으로!
class C:
    pass

c = C()
c.a = 4
d = {}#일반 사전 객체 생성
print d

d[1] = c#실제 객체에 대한 일반 참조 아이템 생성
print d.items()#사전 내용 확인
print d[1].a#실제 객체의 속성 참조

del c#객체 삭제. 사전에 해당 객체의 레퍼런스가 있으므로,
#객체는 실제로 메모리 해제되지 않음
print d.items()#일반 사전에 해당 객체 아이템이 여전히 남아 있음

#화살표를 안지우는 것으로 이해해도 되려나......

{}
[(1, <__main__.C instance at 0x00000000039A7A88>)]
4
[(1, <__main__.C instance at 0x00000000039A7A88>)]


###반복자(Iterator)
- 반복자 객체
  - next()메소드를 지니고 있는 객체
  - next()메소드로 더 이상 자료를 넘겨줄 수 없을 때 StopIteration 예외가 발생한다.
- 반복자 객체 생성 방법
  - iter(o) 내장함수
    - 객체 o의 반복자 객체를 반환한다
- 반복자 객체의 효율성
  - 반복자가 원 객체의 원소들을 복사하여 지니고 있지 않다.

In [1]:
I = iter([1,2,3])
#iter()함수에서 반환되어지는 I가 반복자 객체
print I
#listiterartor객체 호출 첫번째 next()->1, 두번째 ->2, 세번째 ->3
print I.next()
print I.next()
print I.next()
print I.next()
#마지막 I.next()는 가져올 수 있는 원소가 없어서 StopIteration 호출
#I는 각 리스트의 원소를 뽑아낼 수 있는 기능만 있고 카피한 것은 아님
#I자체가 1,2,3내용 자체를 카피해서 가지고 있는 것이 아님.

1
2
3


StopIteration: 

In [3]:
#리스트 객체에 대해 일반적인 for~in 반복문 사용 예
def f(x):
    print x+1
for x in [1,2,3]:
    f(x)

2
3
4


In [4]:
#리스트 객체 반복자를 활용한 예
def f(x):
    print x + 1

t = iter([1,2,3])#반복자 객체는 next()라는 메소드를 가짐
while 1:
    try:
        x = t.next()#더이상 갈 수 없을 때에는 StopIteration발생
    except StopIteration:
        break
    #예외가 발생하지 않으면 try, except내부 내용은 무시됨.
    f(x)

2
3
4


In [7]:
def f(x):
    print x + 1

#반복자 객체를 for~in구문의 in 뒤에 바로 사용 가능
t = iter([1,2,3])
for x in t:#t가 가지고 있는 next()메소드도 자동으로 호출
    f(x)
    #for문이 돌 때 마다 반복자 객체의 next()함수가 자동으로 호출되어 
    #순차적으로 각 객체에 접근 가능하다
    #StopIteration이 발생하면 for~in구문이 멈춘다.
print

#반복자 객체를 정의하지 않고 바로 iter를 이용하여 직접 사용도 가능
for x in iter([1,2,3]):
    f(x)
for x in iter((1,2,3)):#iter의 객체로 리스트가 아닌 튜플도 가능
    f(x)

#반복자는 이후에 발생자를 배우기 위해 배우는 것임

2
3
4

2
3
4


##클래스에 반복자 구현하기
- 내장 함수 iter(o)에 대응되는 \_\_iter\_\_(self)의 구현
- 객체 o에 iter(o)를 호출하면 자동으로 \_\_iter\_\_(self) 함수 호출
- \_\_iter\_\_(self) 함수는 next() 함수를 지닌 반복자 객체를 반환해야 한다.

In [8]:
class Seq:
    def __init__(self, fname):
        self.file = open(fname)
    #def __getitem__(self, n):
    #    if n == 10:
    #        raise StopIteration
    #    return n
    def __iter__(self):#iter 내장함수와 매칭
        return self#iter 내장함수는 반복자 객체(자기자신)를 리턴
    def next(self):#self가 바로 이 next를 가지고 있기 때문에
        #자기 자신을 반환할 수 있는 것ㅇ
        line = self.file.readline()#한 라인을 읽는다.
        if not line: 
            raise StopIteration#읽을 수 없으면 예외 발생
        return line#읽은 라인을 리턴한다.
    
s = Seq('readme.txt')#s 인스턴스가 next() 메소드를 지니고 있으므로,
# s 인스턴스 자체가 반복자임 

for line in s:#s객체는 사용자가 생성한 클래스의 객체
    #so, s에는 반드시 __iter__이 존재해야 함->없으면 에러 발생
    #for~in 구문에 s가 들어가면 iter 내장함수 호출 됨
    #우선 __iter__() 메소드를 호출하여 반복자를 얻고, 
    #반복자에 대해서 for ~ in 구문에 의하여 next() 메소드가 호출됨
    #(self의 next가......)
    print line,

#abc, 그 다음에 for in을 돌면서 readlines를 읽으면서 줄을 읽어오고,
#반환된것이 line에 들어가서 실행 됨.
print 

print Seq('readme.txt')

print list(Seq('readme.txt')#객체를 만드는 곳
#내장함수임과 동시에 리스트 클래스의생성자
#list()내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 
#next()를 매번 호출하여 각 원소를 얻어온다. 
print tuple(Seq('readme.txt'))
#tuple()내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 
#next()를 매번 호출하여 각 원소를 얻어온다. 
#tuple도 리스트와 마찬가지. iter라는 함수 호출하고 self를 받으니까
#self의 next가 호출되면서......


#일단은 iter를 불러준 뒤 반복자를 가져옴 -> next 메소드 호출
#for x in (): 구문의 경우 반복자 객체 self로 리스트 내의 
#내용을 받아와서 x에 담아감 

abc
def
ghi
<__main__.Seq instance at 0x0000000003ADEB88>
['abc\n', 'def\n', 'ghi']
('abc\n', 'def\n', 'ghi')


In [10]:
d = {'one':1, 'two':2, 'three':3, 'four':4, 'five':5}
for key in d#d에 대해서 먼저 iter 함수가 불림
    print key, d[key]

four 4
three 3
five 5
two 2
one 1


In [12]:
d = {'one':1, 'two':2, 'three':3, 'four':4, 'five':5}
for key in iter(d):#실제로는 이것과 같이 iter가 불려지고 있는 것임
    #iter가 불려져서 반복자를 가짐, 반복자는 next를 가짐
    #next()->각각의 아이템의 key값만 돌려줌
    print key, d[key]

#for~in에 쓰여지는 객체는 반드시 반복자 객체를 가짐

four 4
three 3
five 5
two 2
one 1


In [16]:
#iterkeys()는 key값만 가져옴
for key in d.iterkeys():
    #키에 대한 반복자, d.iterkeys() 가 반환한 반복자에 대해 next() 함수가 순차적으로 불리워짐
    print key,
    

four three five two one


In [15]:
#itervalues()->value값만 가져옴
for value in d.itervalues():#값에 대한 반복자
    print value, 
print
for key, value in d.iteritems():#(키,값)에 대한 반복자
    print key, value

4 3 5 2 1
four 4
three 3
five 5
two 2
one 1


In [17]:
#T's
d = {'one':1, 'two':2, 'three':3, 'four':4, 'five':5}
keyset = d.iterkeys()#iterkeys()->key값만 가져와서 반복하는 반복자
print keyset.next()#일단 next를 한번 부름
#반복자 객체는 항상 next() 메소드를 지니고 있음
for key in keyset:
    #keyset 반복자에 대해 next() 메소드가 순차적으로 호출됨
    print key,

#처음의 four은 네번째의 next에서 찍히는 것
#그 밑에 것들은 for문에서 찍히는 것임

#next가 한 번 호출되면 반복자는 그 다음 next는 다음 원소 활용

four
three five two one


In [18]:
#파일 객체는 그 자체가 반복자임
#next()함수에 의해 각 라인이 순차적으로 읽혀짐
f = open('readme.txt')
print "f.next()", f.next()#readme의 abc가 찍힘
#파일 객체 자체가 반복자 -> next를 가지고 있기 때문
for line in f:#f.next() 가 순차적으로 호출됨
    print line,
    #콤마가 있어도 옆에 같이 출력되지 않는 이유는 개행문자 때문임

f.next() abc

def
ghi


- 발생자(Generator)

  - (중단됨 시점부터) 재실행 가능한 함수
- 아래 함수 f()는 자신의 인수 및 내부 변수로서 a, b, c, d를 지니고 있다.

  - 이러한 a, b, c, d 변수들은 함수가 종료되고 반환될 때 모두 사라진다.
- 발생자는 f()와 같이 함수가 종료될 때 메모리에서 해제되는 것을 막고 다시 함수가 호출 될 때 이전에 수행이 종료되었던 지점 부터 계속 수행이 가능하도록 구현된 함수이다.

In [1]:
def f(a,b):
    c = a * b
    d = a + b
    return c, d
#f함수는 리턴이 되면 a,b,c,d모두 사라짐

- yield 키워드
  - return 대신에 yield에 의해 값을 반환하는 함수는 발생자이다.
  - yield는 return과 유사하게 임의의 값을 반환하지만 함수의 실행 상태를 보존하면서 함수를 호출한 쪽으로 복귀시켜준다.
- 발생자는 곧 반복자이다.
  - 즉, 발생자에게 next() 호출이 가능하다.

###발생자를 만드려면 return 대신 yield!

In [2]:
def generate_ints(N):#발생자 함수
    for i in range(N):
        yield i

In [8]:
gen = generate_ints(3)#gen에는 yield가 반환해주는 값인 i가 들어옴
#발생자 객체를 얻는다. generate_ints() 함수에 대한 초기 스택 프레임이 만들어지나 실행은 중단되어 있는 상태임
print gen#발생자 객체 자체!
#기본적으로는 gen에는 generate_ints 자체의 객체가 들어감
print gen.next()#발생자가 곧 반복자
#발생자 객체는 반복자 인터페이스를 가진다. 발생자의 실행이 재개됨. yield에 의해 값 반환 후 다시 실행이 중단됨
print gen.next() 
#발생자 실행 재개. yield에 의해 값 반환 후 다시 중단
print gen.next() 
#발생자 실행 재개. yield에 의해 값 반환 후 다시 중단
print gen.next() 
#발생자 실행 재개. 
#yield에 의해 더 이상 반환할 값이 없다면 StopIteration 예외를 던짐

<generator object generate_ints at 0x0000000003AD1990>
0
1
2


StopIteration: 

In [4]:
for i in generate_ints(5):
    #stopiteration예외 발생시 for~in구문 빠져나감
    print i,
#for~in구문에 발생자 바로 사용 가능 
#why?발생자는 곧 반복자(next를 가짐!)

0 1 2 3 4


- 발생자 함수와 일반 함수의 차이점
  - 일반 함수는 함수가 호출되면 그 함수 내부에 정의된 모든 일을 마치고 결과를 반환함
  - 발생자 함수는 함수 내에서 수행 중에 중간 결과 값을 반환할 수 있음
- 발생자가 유용하게 사용되는 경우
  - 함수 처리의 중간 결과를 다른 코드에서 참조할 경우
  - 모든 결과를 한꺼번에 처리하는 것이 아니라 함수 처리 중에 나온 중간 결과를 사용해야 할 경우

###발생자 구문
- 리스트 내포(List Comprehension)
  - 리스트 객체의 새로운 생성
  - 메모리를 실제로 점유하면서 생성됨

In [5]:
print [k for k in range(100) if k % 5 == 0]#k는 리스트의 원소

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]


- 리스트 내포 구문에 []가 아니라 () 사용
  - 리스트 대신에 발생자 생성
  - 처음부터 모든 원소가 생성되지 않고 필요한 시점에 각 원소가 만들어짐
  - 메모리를 보다 효율적으로 사용함

In [6]:
#발생자를 만드는 구문(함수가 아닌 한줄짜리 구문으로......)
a = (k for k in range(100) if k % 5 == 0)
print a
print a.next()
print a.next()
print a.next()
for i in a:
    print i,
    
    
    
#리스트 내포는 리스트 전반적인 내용을 모두 생성(처음부터 모든 원소 생성)
#발생자는 next가 호출될 때마다 리스트 내용을 발생시킴(필요할때 원소 생성)

<generator object <genexpr> at 0x0000000003AD1828>
0
5
10
15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95


- 아래 예는 sum 내장 함수에 발생자를 넣어줌
  - sum을 호출하는 시점에는 발생자가 아직 호출되기 직전이므로 각 원소들은 아직 존재하지 않는다.
  - sum 내부에서 발생자가 지니고 있는 next() 함수를 호출하여 각 원소들을 직접 만들어 활용한다.
  - 메모시 사용 효율이 높다.

In [7]:
print sum((k for k in range(100) if k % 5 == 0))
#sum을 호출하는데 안쪽 인자가 발생자=>k를 generate
#리스트 내포의 경우 전체 내용이 메모리에 존재하는 상황에서 sum......
#but 발생자는 그때그때 만들어가면서......!

950


In [9]:
#발생자 활용 예 1 - 피보나치 수열
def fibonacci(a = 1, b = 1):
    while 1:
        yield a#yield가 있으니까 피보나치는 발생자~
        a, b = b, a + b
        
for k in fibonacci():#발생자를 직접 for ~ in 구문에 활용
    if k > 100: 
        break
    print k,

1 1 2 3 5 8 13 21 34 55 89


In [10]:
#발생자의 활용 예 2 - 홀수 집합 만들기

#반복자를 활용한 예
class Odds:
    def __init__(self, limit = None):#생성자 정의
        self.data = -1#초기 값
        self.limit = limit#한계 값
    def __iter__(self):#Odds 객체의 반복자를 반환하는 특수 함수
        return self
    def next(self):#반복자의 필수 함수
        self.data += 2
        if self.limit and self.limit <= self.data:
            raise StopIteration
        return self.data

for k in Odds(20):#odds라는 클래스에는 인스턴스 메소드로 iter를 가짐
    #반복자를 next를 가지고 있는 self로 가져옴
    print k,
print
print list(Odds(20))
#list() 내장 함수가 객체를 인수로 받으면 해당 객체의 
#반복자를 얻어와 next()를 매번 호출하여 각 원소를 얻어온다. 

1 3 5 7 9 11 13 15 17 19
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]


In [11]:

#발생자를 활용한 예
def odds(limit=None):
    k = 1#초기값
    while not limit or limit >= k:
        yield k
        k += 2
        
for k in odds(20):
    #for~in구문에 odds(20)사용 가능->odds라는 반복자는 곧 발생자
    print k,
print
print list(odds(20))
#list() 내장 함수가 발생자를 인수로 받으면 해당 발생자의 
#next()를 매번 호출하여 각 원소를 얻어온다. 

1 3 5 7 9 11 13 15 17 19
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]


In [15]:
def getOddSquare(limit):
    a = 1
    while a <= limit:
        yield a * a
        a += 2
def getOddSquare2(N):
    for i in range(N):
        if(i%2!=0):
            yield i*i

In [17]:
for k in getOddSquare(10):
    print k,
print
for k in getOddSquare2(10):#...이거도 맞잖ㅇ...
    print k,


1 9 25 49 81
1 9 25 49 81


In [1]:
class Seq:
    def __init__(self, fname):#우선순위 종류!
        self.file = open(fname)
    #def __getitem__(self, n):->우선순위가 별로 높지 않음
    #    if n == 10:
    #        raise StopIteration
    #    return n
    #ㅁa
    def __iter__(self):
        return self
    def next(self):
        line = self.file.readline() # 한 라인을 읽는다.
        if not line: 
            raise StopIteration     # 읽을 수 없으면 예외 발생
        return line                 # 읽은 라인을 리턴한다.
    
s = Seq('readme.txt')     # s 인스턴스가 next() 메소드를 지니고 있으므로 s 인스턴스 자체가 반복자임 
for line in s: # 우선 __iter__() 메소드를 호출하여 반복자를 얻고, 반복자에 대해서 for ~ in 구문에 의하여 next() 메소드가 호출됨
    print line,

print 

print Seq('readme.txt')

print list(Seq('readme.txt'))  # list() 내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 next()를 매번 호출하여 각 원소를 얻어온다. 
print tuple(Seq('readme.txt')) # tuple() 내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 next()를 매번 호출하여 각 원소를 얻어온다. 



#readlines는 파일의 포인터가 이;ㅇ

abc
def
ghi
<__main__.Seq instance at 0x00000000038E7748>
['abc\n', 'def\n', 'ghi']
('abc\n', 'def\n', 'ghi')


반복자의 경우 반드시 iter와 next를 만들어 주어야 함

In [4]:
#keys values items
#iter를 불러서 가져오는 것은 반복자를 ......?"pme.w"eJ
d={'one':'n'}
for value in d.itervalues():    # 값에 대한 반복자
    print value, 
    #메모리 효율 면에서 그냥 value보다 더 나음

n


In [5]:
#파일객체 f는 반복자라 할 수 있음 next가 있으니까.

f = open('readme.txt')
print "f.next()", f.next()
for line in f:# f.next() 가 순차적으로 호출됨
    print line,    

f.next() abc

def
ghi


In [6]:
f = open('readme.txt')
print "f.next()", f.next()#이거때매 첫ㅂ번째 원소는 이미 떨어져 나감
for line in f:   # f.next() 가 순차적으로 호출됨
    print line,    #for문에서는 두번째 원소부터.....

f.next() abc

def
ghi


#generator 객체
1) 함수 객체  
2) yield 키워드로 값을 리턴해주어야 함.  


  =>모든 발생자 객체는 반복자임!!!!!!   
  =>발생자 객체에게 next 호출이 가능함.

In [8]:
def generate_ints(N):
    for i in range(N):
        yield 
     
    #제너레이트 객체는 수행 후에도 없어지지 않음......
    #어느 부분까지 수행했는지를 기억함
    #자바에는 없고 파이썬에만 존재하는 것.or i in generate_ints(5):
    print i

None None None None None 4


In [9]:

for i in generate_ints(5):
    print i,

None None None None None 4


###아래는 외우고 있어야 하는 부분

In [10]:
a = (k for k in range(100) if k % 5 == 0)
print a
print a.next()
print a.next()
print a.next()
for i in a:
    print i,
print [k for k in range(100) if k % 5 == 0]#리스트 넣어준거
print sum([k for k in range(100) if k % 5 == 0])#리스트 있는거면 넣어준거
print sum((k for k in range(100) if k % 5 == 0))#발생자 넣어준거
#이건 함수가 어떤 갑을 하나하나 읽고 이
#but메모리 효율에서 밑에것이 훨씬 좋음.

<generator object <genexpr> at 0x0000000003961870>
0
5
10
15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
950
950


In [ ]:
def fibonacci(a = 1, b = 1):
    while 1:
        yield a 
        a, b = b, a + b
        
for k in fibonacci():  # 발생자를 직접 for ~ in 구문에 활용
    if k > 100: #이 숫자가 매우 커도 실행이 제대로 되네
        break
    print k,
    
    #코드어디에도 메모리 1부터 89사이 숫자들이 저장되지는 않음.
    #숫자 자체를 언제든 가져다 쓰려면 이렇게 만들면 안돼겠지(사라지니)